#### Setup environment

In [23]:
# %load "/Users/haleighwright/Desktop/NCEAS/pyriv/notebooks/nb_config.py"
import sys
sys.path.append("/Users/haleighwright/Desktop/NCEAS/pyriv")
sys.path.append("/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages")
import networkx as nx
import numpy as np
import sys
import geopandas as gpd
from shapely.geometry import Point
from multiprocessing import Pool

from pyriv import graph_prep as GraphBuilder
from pyriv import river_graph as RiverGraph

%matplotlib inline

In [24]:
from functools import partial

### Retrieve the dead ends of the network
The dead ends are the nodes that have an out degree = 0

In [25]:
# to be efficient with this process, we need to assign node ID's - nodes are identified by their coord's when read in

In [26]:
def add_node_ids_copy(G):
    """
    Params: networkx graph
    Return: copy of graph with node ID attr. added
    
    Returns a copy of the graph with node ID attr (an int) added, 
    used for finding problem nodes easier / quicker node identification
    Must use a dict with the nodes' as keys and the attributes listed
    """
    copy = G.copy()
    return add_node_ids_mutate(copy)

In [27]:
def add_node_ids_mutate(G):
    """
    Params: networkx graph
    Return: referfence to modified graph
    
    Modifies input graph with node ID attr (an int) added, 
    used for finding problem nodes easier / quicker node identification
    Must use a dict with the nodes' as keys and the attributes listed
    """
    attr_dict = {}
    i = 0
    keys = G.nodes()
    for node in keys:
        attr_dict[node] = i
        i = i+1
    nx.set_node_attributes(G, 'ID', attr_dict)
    return G

In [28]:
def deadends_old(self, allnodes):
    """
    For a directed graph, find the nodes that have out_degree == 0
    Note: this is the older, slower method
    """
    thedict = {}
    # get a dictionary of { node: out_degree }
    degdict = self.out_degree(self.nodes())
    # convert to a 2 column array of node and out_degree
    degarr = np.array(degdict.items())
    # get a boolean index of rows where out_degree==0
    dead_row_ind = (degarr[:,1]==0)

    
    for i in range(0, len(degarr)):
        if dead_row_ind[i] == True:
            thedict[allnodes[i][0]] = tuple(degarr[:,0][i])
            
    # use that index to get the nodes
    #dead_nodes = degarr[:,0][dead_row_ind]
    return thedict

In [29]:
def find_missing_edges_seq(G, ends, allnodes, threshold):
    """
    Params:
    Returns: list of missing edges

    Sequential method to find missing edges. VERY VERY SLOW; use ||'d version
    """
    for key1, node1 in deadends.iteritems():
        for key2, node2 in allnodes.iteritems():
            #print node1
            xd = node1[0] - node2[0]
            yd = node1[1] - node2[1]
            dist = np.sqrt((xd*xd + yd*yd))
            
            if key1 != key2 and dist < threshold and dist > 0:
                graph.add_edge(key1, key2)
                print("added edge from "+str(key1)+" : "+str(node1)+" to "+ str(key2)+" : "+str(node2))
                problem_points[str(cnt)+"_A"] =key1
                problem_points[str(cnt)+"_B"] =key2
                break
                
        



In [30]:
def find_missing_edges_par(G, ends, allnodes, threshold):
    """
    Params:
    Returns:
    
    ||'d method to find missing edges (a CPU-bound task w/ no communication needs)
    Note: in order to pass multiple args to multiproc.'g map we use a partial (Python >=2.7)
    """
    
    partial_missing_edges = partial(G, ends, allnodes, threshold)
    
    

In [31]:
def find_missing_edges_par_inner(key1):
    """
    inner function for || version
    """
    
    
    

In [32]:
def add_missing_edges():
    

IndentationError: expected an indented block (<ipython-input-32-f04193e812c2>, line 2)

In [33]:
def missing_edges_to_shp(pts, filepath, projection_code):
    """
    Params: nodes/pts describing missing edges; relative filepath .shp to write to; projection_code to convert
            coords (should match the maps you started with for overlaying purposes)
    Returns: none (writes to output file) 
    
    
    """
    pts = list(pts.values())
    kcdf = gpd.GeoDataFrame({'geometry': [Point(n) for n in pts]})
    kcdf.crs = {'init': projection_code}
    kcdf.to_file(filepath)